In [1]:
import json
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
# from sklearn.linear_model import LinearRegression
mpl.rcParams['figure.dpi']= 200
%config InlineBackend.figure_format = 'svg'
# plt.rcParams['font.size'] = 15
# mpl.rcParams['axes.linewidth'] = 2

RYTOEV = 13.605698 # Rydberg to eV
ryd2mev = RYTOEV*1000

In [2]:
def read_ibz2bz(fname):
    m = []
    with open(fname,"r") as f:
        f.readline()
        line = f.readline()
        while line != "":
            m.append(int(line.split(',')[1]))
            line = f.readline()
    return m

def load_equivalence(prefix,nqibz):
    mappings = []
    for iq in range(nqibz):
        fname = prefix+"ibz2bz_%05d.csv"%(iq+1)
        mapping = read_ibz2bz(fname)
        mappings.append(mapping)
    return mappings

In [3]:
# Reference values taken from Ref. 14 of the main text
ref_gamma = [ -12.53,  24.83,  -14.23,  -30.93, -20.98,  -16.60,  10.10, -180.55]
ref_L     = [ -53.73, 181.28, -273.58, -307.54, -89.36, -220.56, -25.91,  163.19]

# EPCE computed with WELPH with Ponce lattice constant

In [4]:
nqibz = 16
nkibz = 16
nk = 6**3
nq = 6**3
fm = pd.read_csv("./60Ry-Ponce-latt/c.welph.save.pdep200/EPCE_DFT_FM.tab",
                   skiprows=1,names=["ikr","ibn","iq","imode","en"],sep="\\s+")
dw = pd.read_csv("./60Ry-Ponce-latt/c.welph.save.pdep200/EPCE_DFT_DW.tab",
                   skiprows=1,names=["ikr","ibn","iq","imode","en"],sep="\\s+")
mappings = load_equivalence("./60Ry-Ponce-latt/",nqibz)

In [6]:
## EPCE computed with Ponce lattice structure 
my_ponce_gamma = []
my_ponce_L = []

# q = gamma, k = gamma
iqibz = 1
ikibz = 1
filter1 = fm["ikr"] == ikibz
filter2 = fm["iq"].isin(mappings[iqibz-1])
fmpart = fm[filter1 & filter2]
filter3 = dw["ikr"] == ikibz
filter4 = dw["iq"].isin(mappings[iqibz-1])
dwpart = dw[filter3 & filter4]
q_degeneracy = len(mappings[iqibz-1])
tmp_df = fmpart.groupby("ibn")["en"].sum()/q_degeneracy+dwpart.groupby("ibn")["en"].sum()/q_degeneracy
my_ponce_gamma += [tmp_df.values[0],tmp_df.values[1],tmp_df.values[4],tmp_df.values[7]]

# q = gamma, k = L
iqibz = 1
ikibz = 4
filter1 = fm["ikr"] == ikibz
filter2 = fm["iq"].isin(mappings[iqibz-1])
fmpart = fm[filter1 & filter2]
filter3 = dw["ikr"] == ikibz
filter4 = dw["iq"].isin(mappings[iqibz-1])
dwpart = dw[filter3 & filter4]
q_degeneracy = len(mappings[iqibz-1])  
tmp_df = fmpart.groupby("ibn")["en"].sum()/q_degeneracy+dwpart.groupby("ibn")["en"].sum()/q_degeneracy
my_ponce_gamma += [tmp_df.values[0],tmp_df.values[1],tmp_df.values[3],tmp_df.values[4]]

# q = L, k = gamma
iqibz = 4
ikibz = 1
filter1 = fm["ikr"] == ikibz
filter2 = fm["iq"].isin(mappings[iqibz-1])
fmpart = fm[filter1 & filter2]
filter3 = dw["ikr"] == ikibz
filter4 = dw["iq"].isin(mappings[iqibz-1])
dwpart = dw[filter3 & filter4]
q_degeneracy = len(mappings[iqibz-1])
tmp_df = fmpart.groupby("ibn")["en"].sum()/q_degeneracy+dwpart.groupby("ibn")["en"].sum()/q_degeneracy
my_ponce_L += [tmp_df.values[0],tmp_df.values[1],tmp_df.values[4],tmp_df.values[7]]

# q = L, k = L
iqibz = 4
ikibz = 4
filter1 = fm["ikr"] == ikibz
filter2 = fm["iq"].isin(mappings[iqibz-1])
fmpart = fm[filter1 & filter2]
filter3 = dw["ikr"] == ikibz
filter4 = dw["iq"].isin(mappings[iqibz-1])
dwpart = dw[filter3 & filter4]
q_degeneracy = len(mappings[iqibz-1])  
tmp_df = fmpart.groupby("ibn")["en"].sum()/q_degeneracy+dwpart.groupby("ibn")["en"].sum()/q_degeneracy
my_ponce_L += [tmp_df.values[0],tmp_df.values[1],tmp_df.values[3],tmp_df.values[4]]

# EPCE computed with WELPH with my relaxed lattice constant

In [7]:
nqibz = 16
nkibz = 16
nk = 6**3
nq = 6**3
fm = pd.read_csv("./60Ry-myrelax/c.welph.save.pdep200/EPCE_DFT_FM.tab",
                   skiprows=1,names=["ikr","ibn","iq","imode","en"],sep="\\s+")
dw = pd.read_csv("./60Ry-myrelax/c.welph.save.pdep200/EPCE_DFT_DW.tab",
                   skiprows=1,names=["ikr","ibn","iq","imode","en"],sep="\\s+")
mappings = load_equivalence("./60Ry-myrelax/",nqibz)

In [10]:
## EPCE computed with my own relaxed lattice structure 
my_relax_gamma = []
my_relax_L = []

# q = gamma, k = gamma
iqibz = 1
ikibz = 1
filter1 = fm["ikr"] == ikibz
filter2 = fm["iq"].isin(mappings[iqibz-1])
fmpart = fm[filter1 & filter2]
filter3 = dw["ikr"] == ikibz
filter4 = dw["iq"].isin(mappings[iqibz-1])
dwpart = dw[filter3 & filter4]
q_degeneracy = len(mappings[iqibz-1])
tmp_df = fmpart.groupby("ibn")["en"].sum()/q_degeneracy+dwpart.groupby("ibn")["en"].sum()/q_degeneracy
my_relax_gamma += [tmp_df.values[0],tmp_df.values[1],tmp_df.values[4],tmp_df.values[7]]

# q = gamma, k = L
iqibz = 1
ikibz = 4
filter1 = fm["ikr"] == ikibz
filter2 = fm["iq"].isin(mappings[iqibz-1])
fmpart = fm[filter1 & filter2]
filter3 = dw["ikr"] == ikibz
filter4 = dw["iq"].isin(mappings[iqibz-1])
dwpart = dw[filter3 & filter4]
q_degeneracy = len(mappings[iqibz-1])  
tmp_df = fmpart.groupby("ibn")["en"].sum()/q_degeneracy+dwpart.groupby("ibn")["en"].sum()/q_degeneracy
my_relax_gamma += [tmp_df.values[0],tmp_df.values[1],tmp_df.values[3],tmp_df.values[4]]

# q = L, k = gamma
iqibz = 4
ikibz = 1
filter1 = fm["ikr"] == ikibz
filter2 = fm["iq"].isin(mappings[iqibz-1])
fmpart = fm[filter1 & filter2]
filter3 = dw["ikr"] == ikibz
filter4 = dw["iq"].isin(mappings[iqibz-1])
dwpart = dw[filter3 & filter4]
q_degeneracy = len(mappings[iqibz-1])
tmp_df = fmpart.groupby("ibn")["en"].sum()/q_degeneracy+dwpart.groupby("ibn")["en"].sum()/q_degeneracy
my_relax_L += [tmp_df.values[0],tmp_df.values[1],tmp_df.values[4],tmp_df.values[7]]

# q = L, k = L
iqibz = 4
ikibz = 4
filter1 = fm["ikr"] == ikibz
filter2 = fm["iq"].isin(mappings[iqibz-1])
fmpart = fm[filter1 & filter2]
filter3 = dw["ikr"] == ikibz
filter4 = dw["iq"].isin(mappings[iqibz-1])
dwpart = dw[filter3 & filter4]
q_degeneracy = len(mappings[iqibz-1])  
tmp_df = fmpart.groupby("ibn")["en"].sum()/q_degeneracy+dwpart.groupby("ibn")["en"].sum()/q_degeneracy
my_relax_L += [tmp_df.values[0],tmp_df.values[1],tmp_df.values[3],tmp_df.values[4]]

# Summary of the final results

In [11]:
for x, y, z in zip(my_relax_gamma,my_ponce_gamma,ref_gamma):
    print(f"{x:8.2f}{y:8.2f}{z:8.2f}")

  -12.55  -12.84  -12.53
   25.13   24.86   24.83
  -14.87  -14.88  -14.23
  -31.91  -30.86  -30.93
  -21.18  -21.54  -20.98
  -16.72  -16.91  -16.60
   10.14   10.02   10.10
 -162.66 -182.88 -180.55


In [12]:
for x, y, z in zip(my_relax_L,my_ponce_L,ref_L):
    print(f"{x:8.2f}{y:8.2f}{z:8.2f}")

  -54.47  -55.15  -53.73
  186.17  186.71  181.28
 -273.16 -274.86 -273.58
 -311.90 -294.70 -307.54
  -91.27  -91.96  -89.36
 -212.64 -224.15 -220.56
  -26.88  -27.13  -25.91
  163.96  164.07  163.19


In [24]:
my_relax_mad = np.mean(np.abs(np.array(my_relax_gamma + my_relax_L) - np.array(ref_gamma + ref_L)))
my_ponce_mad = np.mean(np.abs(np.array(my_ponce_gamma + my_ponce_L) - np.array(ref_gamma + ref_L)))
my_relax_mard = np.mean(np.abs(np.array(my_relax_gamma + my_relax_L)/np.array(ref_gamma + ref_L)-1.0))
my_ponce_mard = np.mean(np.abs(np.array(my_ponce_gamma + my_ponce_L)/np.array(ref_gamma + ref_L)-1.0))

print(f"{my_relax_mad:8.2f}{my_ponce_mad:8.2f}")
print(f"{my_relax_mard*100:7.2f}%{my_ponce_mard*100:7.2f}%")

    2.64    2.10
   2.29%   2.13%
